<a href="https://colab.research.google.com/github/alexmoed/MasterClass/blob/3D-Pointnet%2B%2B/PointNet_Classification_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
 # !git clone https://github.com/karol-202/direct-3dgs-segmentation "/content/drive/MyDrive/3dgs/"

In [19]:
from google.colab import drive
drive.mount('/content/drive')


#change the directory to the correct spot on google drive"""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Train a semantic segmentation model for 3D Gaussian Splats
# Modified from :
# Yanx27 (2019). PointNet_Pointnet2_pytorch [online].
# [Accessed 2024]. Available from: "https://github.com/yanx27/Pointnet_Pointnet2_pytorch"
# Based on research from:
# Jurski, K. (2024). Semantic 3D segmentation of 3D Gaussian Splats: Assessing existing
# point cloud segmentation techniques on semantic segmentation of synthetic 3D Gaussian
# Splats scenes. Bachelor's Thesis, Delft University of Technology.
# Extended implementation: "https://github.com/karol-202/direct-3dgs-segmentation"

##PointNet ++ for Gaussian splats

Installing depencencies

In [20]:
!pip install plyfile

In [21]:
#Uninstall any versions of pytorch3d
!pip uninstall -y torch torchvision torchaudio pytorch3d


Found existing installation: torch 2.6.0.dev20241112+cu121
Uninstalling torch-2.6.0.dev20241112+cu121:
  Successfully uninstalled torch-2.6.0.dev20241112+cu121
Found existing installation: torchvision 0.20.0.dev20241112+cu121
Uninstalling torchvision-0.20.0.dev20241112+cu121:
  Successfully uninstalled torchvision-0.20.0.dev20241112+cu121
Found existing installation: torchaudio 2.5.0.dev20241112+cu121
Uninstalling torchaudio-2.5.0.dev20241112+cu121:
  Successfully uninstalled torchaudio-2.5.0.dev20241112+cu121
Found existing installation: pytorch3d 0.7.8
Uninstalling pytorch3d-0.7.8:
  Successfully uninstalled pytorch3d-0.7.8


In [22]:
#Run this exact version of torch vision others dont seem to work with pytorch 3d
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
  Using cached https://download.pytorch.org/whl/nightly/cu121/torch-2.6.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (768.0 MB)
  Using cached https://download.pytorch.org/whl/nightly/cu121/torchvision-0.20.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (7.4 MB)
  Using cached https://download.pytorch.org/whl/nightly/cu121/torchaudio-2.5.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)


In [1]:
!pip install -U cmake ninja
!pip install fvcore iopath


In [2]:
#Install the newest stable version of pytorch3d It takes awhile this is normal
!pip install "git+https://github.com/facebookresearch/pytorch3d.git@stable"
#It will get stuck on building wheels


  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-snfrurta
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-snfrurta
  Running command git checkout -q 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch3d: filename=pytorch3d-0.7.8-cp311-cp311-linux_x86_64.whl size=60074661 sha256=d9d07321b6363a11a2148cb24e240cec47336365080576f7c0da3dfdafbe1c49
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ikemka0/wheels/08/90/1b/df18c3e3634f86278e793b87f37ea4c58d0c36731196122518
Successfully built pytorch3d


In [25]:
import pytorch3d
print(pytorch3d.__version__)
#make sure its the following version
#0.7.8

0.7.8


In [3]:
!pip install trimesh

In [4]:
import torch
print(torch.__version__)

2.6.0.dev20241112+cu121


In [5]:
# Partially from: https://github.com/yanx27/Pointnet_Pointnet2_pytorch
import argparse
import os
import datetime
import logging
from pathlib import Path
import sys
import importlib
import time
import numpy as np
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.utils.data.sampler as sampler

# Add this patch to fix NumPy issue with PyTorch DataLoader
import torch.utils.data.sampler as sampler
# Add this patch to fix NumPy issue with PyTorch DataLoader
import torch.utils.data.sampler as sampler

%cd /content/drive/MyDrive/3dgs
#Change to your repo

# Custom imports
from data_utils.extra_feature import ExtraFeature
from datasets.base_3dgs_dataset import Base3DGSDataset
from datasets.composed_mesh_dataset import ComposedMeshDataset
from datasets.composed_3dgs_dataset import Composed3DGSDataset



/content/drive/MyDrive/3dgs


# Define paths


In [6]:
# Define paths
BASE_DIR = "/content/drive/MyDrive/3dgs"  # Path for Google Colab
experiment_dir = BASE_DIR
ROOT_DIR = BASE_DIR
dataset_path = "/content/drive/MyDrive/3dgs/datasets"  # Path to datasets directory
data_path = "/content/drive/MyDrive/3dgs"  # Path to where train.txt and test.txt are located
test_txt = data_path + "/data/test.txt"

#Data Processing and Class Configuration

In [13]:

#Function to save point clouds with class IDs
def save_classified_pointcloud(points, class_ids, filename):
    """Save point cloud with class ID for each point."""
    with open(filename, 'w') as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property int class_id\n")
        f.write("end_header\n")

        for i in range(len(points)):
            x, y, z = points[i]
            class_id = int(class_ids[i])
            f.write(f"{x} {y} {z} {class_id}\n")

#Read class names from train.txt/test.txt first
def get_classes():
    delimiter = '/'
    test_txt

    #Read test file and extract class names
    categories = pd.read_csv(test_txt, delimiter=delimiter, header=None, names=["col1", "col2"])
    categories_set = set(categories["col1"])  # Get unique class names
    sorted_classes = sorted(categories_set)  # Sort for consistency

    print(f"Sorted Categories from Dataset: {sorted_classes}")
    return sorted_classes

#Set up class names and mappings
CLASSES = get_classes()
class2label = {cls: i for i, cls in enumerate(CLASSES)}
CLASS2LABEL = class2label
NUM_CLASSES = len(CLASSES)
print(f"Class to label mapping: {CLASS2LABEL}")

sys.path.append(os.path.join(ROOT_DIR, 'models'))

seg_classes = class2label
seg_label_to_cat = {}
for i, cat in enumerate(seg_classes.keys()):
    seg_label_to_cat[i] = cat


def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace = True


#Define ExtraFeature class if not imported from elsewhere
class ExtraFeature:
    @staticmethod
    def feature_by_name(name):
        # Simple placeholder implementation
        return name


#Constructing the paths
def modify_paths_train():
    delimiter = '/'
    train_txt = data_path + "/data/train.txt"

    # Read train file and split into two columns
    train_df = pd.read_csv(train_txt, delimiter=delimiter, header=None, names=["col1", "col2"])

    # Construct paths to point cloud files
    train_df["combined"] = dataset_path + "/" + train_df["col1"] + "/" + train_df["col2"] + "/point_cloud/iteration_15000/point_cloud.ply"

    return (train_df["combined"].tolist(), train_df["col1"].tolist())

def modify_paths_test():
    delimiter = '/'
    test_txt = data_path + "/data/test.txt"

    #Read test file and split into two columns
    test_df = pd.read_csv(test_txt, delimiter=delimiter, header=None, names=["col1", "col2"])

    #Construct paths to point cloud files
    test_df["combined"] = dataset_path + "/" + test_df["col1"] + "/" + test_df["col2"] + "/point_cloud/iteration_15000/point_cloud.ply"

    return (test_df["combined"].tolist() , test_df["col1"].tolist())


Sorted Categories from Dataset: ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
Class to label mapping: {'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


In [8]:
#Define all extra feature constants BEFORE get_args()
FEATURE_ROTATION_QUAT = 'rotation_quat'
FEATURE_ROTATION_MATRIX = 'rotation_matrix'
FEATURE_SCALE = 'scale'
FEATURE_COVARIANCE = 'covariance'
FEATURE_OPACITY = 'opacity'
FEATURE_COLOR = 'color'
FEATURE_REST = 'rest'


In [14]:
#Sanity test making sure the labels and paths have the same lenght meaning theres a l
train_paths, trainlabels = modify_paths_train()
test_paths, testlabels = modify_paths_test()
testlabels


if len(train_paths) == len(trainlabels) and len(test_paths) == len(testlabels):
  print("both train and testing paths are the same lenght as the lables")
else:
    print("PATHS ARE NOT SAME LENGHT")

both train and testing paths are the same lenght as the lables


 ## Setup and Configuration

In [15]:

# Random Sampling
original_iter = sampler.RandomSampler.__iter__
def patched_iter(self):
    n = len(self.data_source)
    if self.generator is None:
        generator = torch.Generator()
        generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
    else:
        generator = self.generator

    if self.replacement:
        for _ in range(self.num_samples // n):
            for idx in torch.randint(0, n, size=(n,), generator=generator).tolist():
                yield idx
        for idx in torch.randint(0, n, size=(self.num_samples % n,), generator=generator).tolist():
            yield idx
    else:

        for idx in torch.randperm(n, generator=generator).tolist():
            yield idx

sampler.RandomSampler.__iter__ = patched_iter

# Function to save point clouds with class IDs
def save_classified_pointcloud(points, class_ids, filename):
    """Save point cloud with class ID for each point."""
    with open(filename, 'w') as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property int class_id\n")
        f.write("end_header\n")

        for i in range(len(points)):
            x, y, z = points[i]
            class_id = int(class_ids[i])
            f.write(f"{x} {y} {z} {class_id}\n")

# Read class names from train.txt/test.txt first
def get_classes():
    delimiter = '/'
    # Read test file and extract class names
    categories = pd.read_csv(test_txt, delimiter=delimiter, header=None, names=["col1", "col2"])
    categories_set = set(categories["col1"])  # Get unique class names
    sorted_classes = sorted(categories_set)  # Sort for consistency

    print(f"Sorted Categories from Dataset: {sorted_classes}")
    return sorted_classes

# Set up class names and mappings
CLASSES = get_classes()
class2label = {cls: i for i, cls in enumerate(CLASSES)}
CLASS2LABEL = class2label
NUM_CLASSES = len(CLASSES)
print(f"Class to label mapping: {CLASS2LABEL}")

seg_classes = class2label
seg_label_to_cat = {}
for i, cat in enumerate(seg_classes.keys()):
    seg_label_to_cat[i] = cat

def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace = True

# Define ExtraFeature class if not imported from elsewhere
class ExtraFeature:
    @staticmethod
    def feature_by_name(name):
        # Simple placeholder implementation
        return name


class TrainEnv:
    def log_string(self, str):
        self.logger.info(str)
        print(str)

def create_environment(args, train_paths, test_paths, train_labels, test_labels):
    env = TrainEnv()

    '''HYPER PARAMETER'''
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    '''CREATE DIR'''
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    experiment_dir = Path('./log/')
    experiment_dir.mkdir(exist_ok=True)
    experiment_dir = experiment_dir.joinpath('sem_seg')
    experiment_dir.mkdir(exist_ok=True)
    if args.log_dir is None:
        experiment_dir = experiment_dir.joinpath(timestr)
    else:
        experiment_dir = experiment_dir.joinpath(args.log_dir)
    experiment_dir.mkdir(exist_ok=True)
    env.checkpoints_dir = experiment_dir.joinpath('checkpoints/')
    env.checkpoints_dir.mkdir(exist_ok=True)
    log_dir = experiment_dir.joinpath('logs/')
    log_dir.mkdir(exist_ok=True)

    '''LOG'''
    env.logger = logging.getLogger("Model")
    env.logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler = logging.FileHandler('%s/%s.txt' % (log_dir, args.model))
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    env.logger.addHandler(file_handler)
    env.log_string('PARAMETER ...')
    env.log_string(args)

    env.writer = SummaryWriter(log_dir=str(experiment_dir.joinpath('tensorboard')))

    num_point = args.npoint
    batch_size = args.batch_size

    sampling = args.sampling

    print("start loading training data ...")
    if args.dataset_type == '3DGS':
        from datasets.composed_3dgs_dataset import Composed3DGSDataset

        env.train_dataset = Composed3DGSDataset(
            model_paths=train_paths,
            class2label=CLASS2LABEL,  # Using class2label for label mapping
            sampling=sampling,
            num_point=num_point,
            extra_features=args.extra_features  # Pass the `ExtraFeature` objects here
        )
    elif args.dataset_type == 'SampledMesh':
        from datasets.composed_mesh_dataset import ComposedMeshDataset

        env.train_dataset = ComposedMeshDataset(model_paths=train_paths, class2label=CLASS2LABEL, num_point=num_point)

    print("start loading test data ...")
    if args.dataset_type == '3DGS':
        env.test_dataset = Composed3DGSDataset(
            model_paths=test_paths,
            class2label=CLASS2LABEL,  # Using class2label for label mapping
            sampling=sampling,
            num_point=num_point,
            extra_features=args.extra_features  # Pass the `ExtraFeature` objects here
        )
    elif args.dataset_type == 'SampledMesh':
        env.test_dataset = ComposedMeshDataset(model_paths=test_paths, class2label=CLASS2LABEL, num_point=num_point)

    def custom_worker_init_fn(worker_id):
        # Use PyTorch's random number generator instead of NumPy's
        worker_seed = torch.initial_seed() % 2**32
        torch.manual_seed(worker_seed)
        # Only set NumPy seed if it's safe to do so
        try:
            np.random.seed(worker_seed)
        except:
            pass

    env.trainDataLoader = torch.utils.data.DataLoader(
        env.train_dataset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=False,  # Disabled pin_memory
        drop_last=True,
        num_workers=4,
        worker_init_fn=custom_worker_init_fn
    )

    env.testDataLoader = torch.utils.data.DataLoader(
        env.test_dataset,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=False,  # Disabled pin_memory
        drop_last=True,
        num_workers=4,
        worker_init_fn=custom_worker_init_fn
    )

    env.weights = torch.Tensor(env.train_dataset.label_weights).cuda()

    env.log_string("The number of training data is: %d" % len(env.train_dataset))
    env.log_string("The number of test data is: %d" % len(env.test_dataset))

    '''MODEL LOADING'''
    MODEL = importlib.import_module(args.model)

    env.classifier = MODEL.get_model(NUM_CLASSES, env.train_dataset.get_channels_count).cuda()
    env.classifier.apply(inplace_relu)
    env.criterion = MODEL.get_loss().cuda()

    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv2d') != -1:
            torch.nn.init.xavier_normal_(m.weight.data)
            torch.nn.init.constant_(m.bias.data, 0.0)
        elif classname.find('Linear') != -1:
            torch.nn.init.xavier_normal_(m.weight.data)
            torch.nn.init.constant_(m.bias.data, 0.0)

    try:
        checkpoint = torch.load(str(experiment_dir) + '/checkpoints/model.pth')
        env.start_epoch = checkpoint['epoch'] + 1
        env.classifier.load_state_dict(checkpoint['model_state_dict'])
        env.log_string('Use pretrain model')
    except:
        env.log_string('No existing model, starting from scratch...')
        env.start_epoch = 0
        env.classifier = env.classifier.apply(weights_init)

    if args.optimizer == 'Adam':
        env.optimizer = torch.optim.Adam(
            env.classifier.parameters(),
            lr=args.learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08,
            weight_decay=args.weight_decay_rate
        )
    else:
        env.optimizer = torch.optim.SGD(env.classifier.parameters(), lr=args.learning_rate, momentum=0.9)

    return env


def close_environment(env):
    env.writer.close()

    handlers = env.logger.handlers[:]
    for handler in handlers:
        env.logger.removeHandler(handler)
        handler.close()
        #Show classes

Sorted Categories from Dataset: ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
Class to label mapping: {'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


PointNet segmentation code I modified, with help from Claude AI
I took the original PointNet segmentation code and made several improvements:
- Streamlined how variables are declared for better organization
- Added functionality to save point cloud visualization as PLY files
- Created the EvalResults class for cleaner metric tracking

Claude helped me troubleshoot the PLY file saving implementation and
suggested refinements to the visualization approach. The core structure
is from PointNet, but my modifications make it more usable for my needs.

##Declare your aguments here

In [11]:
#Change your arguments here
def get_args():
    args = argparse.Namespace()

    args.data_path = "/content/drive/MyDrive/3dgs/datasets" #Change to your path
    args.model = 'pointnet2_sem_seg'
    args.dataset_type = '3DGS'
    args.batch_size = 8
    args.epoch = 200
    args.learning_rate = 0.003
    args.gpu = '0'
    args.optimizer = 'Adam'
    args.log_dir = 'epochs250_learningrate_003_bs32_v027_001_4096' #rename each time overwise the model will treat the previous version as a checkpoint
    args.weight_decay_rate = 1e-4
    args.npoint = 4096
    args.lr_step_size = 5  # how often it updates
    args.lr_decay = 0.95
    args.eval_after_epoch = True
    args.sampling = 'uniform'

    # Test mode settings this is just to see if it will train and test (not good results)
    args.test_mode = False #I added this so you can test a smaller sample size to see if the labeling runs and goes onto the next section
    args.max_test_samples = 200
    # Predefined extra features
    predefined_extra_features = [
        'rotation_quat',
        'scale',
        'opacity',
    ]

    # Convert to ExtraFeature objects
    from data_utils.extra_feature import ExtraFeature
    args.extra_features = [ExtraFeature.feature_by_name(feature) for feature in predefined_extra_features] if predefined_extra_features else None
    return args


## Training and Evaluation


In [ ]:
def train(env, args):
    def bn_momentum_adjust(m, momentum):
        if isinstance(m, torch.nn.BatchNorm2d) or isinstance(m, torch.nn.BatchNorm1d):
            m.momentum = momentum

    # Set some constants for the learning scheduler
    LEARNING_RATE_CLIP = 1e-5
    MOMENTUM_ORIGINAL = 0.1
    MOMENTUM_DECCAY = 0.5
    MOMENTUM_DECCAY_STEP = args.lr_step_size

    global_epoch = 0
    best_iou = 0

    #Main training loop
    for epoch in range(env.start_epoch, args.epoch):
        env.log_string('**** Epoch %d (%d/%s) ****' % (global_epoch + 1, epoch + 1, args.epoch))

        # Decay learning rate over time
        lr = max(args.learning_rate * (args.lr_decay ** (epoch // args.lr_step_size)), LEARNING_RATE_CLIP)
        env.log_string('Learning rate:%f' % lr)
        for param_group in env.optimizer.param_groups:
            param_group['lr'] = lr

        #Update batch norm momentum - helps stabilize training
        momentum = MOMENTUM_ORIGINAL * (MOMENTUM_DECCAY ** (epoch // MOMENTUM_DECCAY_STEP))
        if momentum < 0.01:
            momentum = 0.01
        print('BN momentum updated to: %f' % momentum)
        env.classifier = env.classifier.apply(lambda x: bn_momentum_adjust(x, momentum))

        env.writer.add_scalar('LR', lr, epoch)

        # Reset metrics for this epoch
        num_batches = len(env.trainDataLoader)
        total_correct = 0
        total_seen = 0
        loss_sum = 0
        env.classifier = env.classifier.train()

        # Process each batch
        for i, (points, target) in tqdm(enumerate(env.trainDataLoader), total=len(env.trainDataLoader), smoothing=0.9):
            env.optimizer.zero_grad()

            # Prep the point cloud data
            points = points.data.numpy()
            points = torch.Tensor(points)
            points, target = points.float().cuda(), target.long().cuda()
            points = points.transpose(2, 1)  # PointNet needs this format

            # Forward pass through model
            seg_pred, trans_feat = env.classifier(points)
            seg_pred = seg_pred.contiguous().view(-1, NUM_CLASSES)

            batch_label = target.view(-1, 1)[:, 0].cpu().data.numpy()
            target = target.view(-1, 1)[:, 0]

            # Calculate loss and update weights
            loss = env.criterion(seg_pred, target, trans_feat, env.weights)
            loss.backward()
            env.optimizer.step()

            # Track how we're doing
            pred_choice = seg_pred.cpu().data.max(1)[1].numpy()
            correct = np.sum(pred_choice == batch_label)
            total_correct += correct
            total_seen += (args.batch_size * args.npoint)
            loss_sum += loss

        # Log training results
        training_loss = loss_sum / num_batches
        training_accuracy = total_correct / float(total_seen)

        env.log_string('Training mean loss: %f' % training_loss)
        env.log_string('Training accuracy: %f' % training_accuracy)
        env.writer.add_scalar('Train loss', training_loss, epoch)
        env.writer.add_scalar('Train accuracy', training_accuracy, epoch)

        # Save model checkpoints regularly
        if epoch % 5 == 0 or epoch == args.epoch - 1:
            env.log_string('Save model...')
            savepath = str(env.checkpoints_dir) + '/model.pth'
            env.log_string('Saving at %s' % savepath)
            state = {
                'epoch': epoch,
                'model_state_dict': env.classifier.state_dict(),
                'optimizer_state_dict': env.optimizer.state_dict(),
            }
            torch.save(state, savepath)
            env.log_string('Saving model....')

        # Evaluation phase
        if args.eval_after_epoch:
            env.log_string('---- EPOCH %03d EVALUATION ----' % (global_epoch + 1))
            eval_results = evaluate(env, args, epoch=epoch)

            #Log Progress
            env.log_string('eval point avg class IoU: %f' % eval_results.mIoU)
            env.log_string('eval point avg class acc: %f' % (
                eval_results.mean_class_accuracy))

            # Show per-class results
            iou_per_class_str = '------- IoU --------\n'
            for l in range(NUM_CLASSES):
                iou_per_class_str += 'class %s weight: %.3f, IoU: %.3f \n' % (
                    seg_label_to_cat[l] + ' ' * (14 - len(seg_label_to_cat[l])),
                    eval_results.labelweights[l - 1],
                    eval_results.class_mIoU[l]
                )

            env.log_string(iou_per_class_str)
            env.log_string('Eval mean loss: %f' % eval_results.loss)
            env.log_string('Eval accuracy: %f' % eval_results.accuracy)

            env.writer.add_scalar('Eval loss', eval_results.loss, epoch)
            env.writer.add_scalar('Eval accuracy', eval_results.accuracy, epoch)
            env.writer.add_scalar('Eval mIoU', eval_results.mIoU, epoch)

            if eval_results.mIoU >= best_iou:
                best_iou = eval_results.mIoU
                env.log_string('Save model...')
                savepath = str(env.checkpoints_dir) + '/best_model.pth'
                env.log_string('Saving at %s' % savepath)
                state = {
                    'epoch': epoch,
                    'class_avg_iou': eval_results.mIoU,
                    'model_state_dict': env.classifier.state_dict(),
                    'optimizer_state_dict': env.optimizer.state_dict(),
                }
                torch.save(state, savepath)
                env.log_string('Saving model....')
            env.log_string('Best mIoU: %f' % best_iou)

        env.writer.flush()
        global_epoch += 1


# Container for all our evaluation metrics
class EvalResults:
    def __init__(self, mIoU, loss, accuracy, labelweights, mean_class_accuracy, class_mIoU):
        self.mIoU = mIoU
        self.loss = loss
        self.accuracy = accuracy
        self.labelweights = labelweights
        self.mean_class_accuracy = mean_class_accuracy
        self.class_mIoU = class_mIoU


def evaluate(env, args, epoch=None):
    with torch.no_grad():
        # Setup tracking variables
        num_batches = len(env.testDataLoader)
        total_correct = 0
        total_seen = 0
        loss_sum = 0
        labelweights = np.zeros(NUM_CLASSES)
        total_seen_class = [0 for _ in range(NUM_CLASSES)]
        total_correct_class = [0 for _ in range(NUM_CLASSES)]
        total_iou_deno_class = [0 for _ in range(NUM_CLASSES)]
        env.classifier = env.classifier.eval()  # Switch to eval mode

        # Save visualization files every 50 epochs
        save_visualization = (epoch is not None) and (epoch % 50 == 0 or epoch == args.epoch - 1)

        # Need these lists for visualization
        if save_visualization:
            all_points = []
            all_predictions = []
            all_ground_truth = []

        # Process test batches
        for i, (points, target) in tqdm(enumerate(env.testDataLoader), total=len(env.testDataLoader), smoothing=0.9):
            original_points = points.data.numpy()

            points = torch.Tensor(original_points)
            points, target = points.float().cuda(), target.long().cuda()
            points = points.transpose(2, 1)

            # Get model predictions
            seg_pred, trans_feat = env.classifier(points)
            pred_val = seg_pred.contiguous().cpu().data.numpy()
            seg_pred = seg_pred.contiguous().view(-1, NUM_CLASSES)

            batch_label = target.cpu().data.numpy()
            target = target.view(-1, 1)[:, 0]
            loss = env.criterion(seg_pred, target, trans_feat, env.weights)
            loss_sum += loss
            pred_val = np.argmax(pred_val, 2)  # Get class predictions
            correct = np.sum((pred_val == batch_label))
            total_correct += correct
            total_seen += (args.batch_size * args.npoint)
            tmp, _ = np.histogram(batch_label, range(NUM_CLASSES + 1))
            labelweights += tmp

            # Per-class metrics
            for l in range(NUM_CLASSES):
                total_seen_class[l] += np.sum((batch_label == l))
                total_correct_class[l] += np.sum((pred_val == l) & (batch_label == l))
                total_iou_deno_class[l] += np.sum(((pred_val == l) | (batch_label == l)))

            # Collect visualization data if needed
            if save_visualization:
                for b in range(original_points.shape[0]):
                    # Just grab XYZ coordinates
                    batch_points = original_points[b, :, :3]
                    batch_pred = pred_val[b]
                    batch_gt = batch_label[b]

                    all_points.append(batch_points)
                    all_predictions.append(batch_pred)
                    all_ground_truth.append(batch_gt)

        # Save colorized point clouds for visualization
        if save_visualization:
            vis_dir = os.path.join(str(env.checkpoints_dir), 'visualizations')
            os.makedirs(vis_dir, exist_ok=True)

            # Combine all the collected data
            full_points = np.vstack(all_points)
            full_predictions = np.concatenate(all_predictions)
            full_ground_truth = np.concatenate(all_ground_truth)

            # Save as PLY files that can be viewed in 3D software
            pred_filename = os.path.join(vis_dir, f'epoch_{epoch}_full_prediction.ply')
            save_classified_pointcloud(full_points, full_predictions, pred_filename)

            gt_filename = os.path.join(vis_dir, f'epoch_{epoch}_full_groundtruth.ply')
            save_classified_pointcloud(full_points, full_ground_truth, gt_filename)

        # Calculate final metrics
        labelweights = labelweights.astype(np.float32) / np.sum(labelweights.astype(np.float32))
        mIoU = np.mean(np.array(total_correct_class) / (np.array(total_iou_deno_class, dtype=float) + 1e-6))
        eval_loss = loss_sum / float(num_batches)
        eval_accuracy = total_correct / float(total_seen)
        mean_class_accuracy = np.mean(np.array(total_correct_class) / (np.array(total_seen_class, dtype=float) + 1e-6))
        class_mIoU = [total_correct_class[l] / float(total_iou_deno_class[l]) for l in range(NUM_CLASSES)]

        return EvalResults(mIoU, eval_loss, eval_accuracy, labelweights, mean_class_accuracy, class_mIoU)

def main():
    # Get the config settings
    args = get_args()

    # Load dataset paths and their labels
    train_paths, trainlabels = modify_paths_train()
    test_paths, testlabels = modify_paths_test()

    # Use fewer samples in test mode
    if args.test_mode:
        print(f"Test mode enabled - using only {args.max_test_samples} samples")
        train_paths = train_paths[:args.max_test_samples]
        trainlabels = trainlabels[:args.max_test_samples]
        test_paths = test_paths[:args.max_test_samples]
        testlabels = testlabels[:args.max_test_samples]

    # Make sure our data is valid
    if len(train_paths) == len(trainlabels) and len(test_paths) == len(testlabels):
        # Setup and run training
        env = create_environment(args, train_paths, test_paths, trainlabels, testlabels)
        train(env, args)
        close_environment(env)
    else:
        # Something's wrong with the data
        raise Exception("The list of labels for the train and test data do not match lengths.")

if __name__ == "__main__":
    main()

INFO:Model:PARAMETER ...
INFO:Model:Namespace(data_path='/content/drive/MyDrive/3dgs/datasets', model='pointnet2_sem_seg', dataset_type='3DGS', batch_size=8, epoch=200, learning_rate=0.003, gpu='0', optimizer='Adam', log_dir='epochs250_learningrate_003_bs32_v027_001_4096', weight_decay_rate=0.0001, npoint=4096, lr_step_size=5, lr_decay=0.95, eval_after_epoch=True, sampling='uniform', test_mode=False, max_test_samples=200, extra_features=[<data_utils.extra_feature.ExtraFeature object at 0x791d469e1790>, <data_utils.extra_feature.ExtraFeature object at 0x791d4734d1d0>, <data_utils.extra_feature.ExtraFeature object at 0x791d4734f5d0>])


PARAMETER ...
Namespace(data_path='/content/drive/MyDrive/3dgs/datasets', model='pointnet2_sem_seg', dataset_type='3DGS', batch_size=8, epoch=200, learning_rate=0.003, gpu='0', optimizer='Adam', log_dir='epochs250_learningrate_003_bs32_v027_001_4096', weight_decay_rate=0.0001, npoint=4096, lr_step_size=5, lr_decay=0.95, eval_after_epoch=True, sampling='uniform', test_mode=False, max_test_samples=200, extra_features=[<data_utils.extra_feature.ExtraFeature object at 0x791d469e1790>, <data_utils.extra_feature.ExtraFeature object at 0x791d4734d1d0>, <data_utils.extra_feature.ExtraFeature object at 0x791d4734f5d0>])
start loading training data ...


  0%|          | 0/3991 [00:00<?, ?it/s]

classname: bathtub, path: /content/drive/MyDrive/3dgs/datasets/bathtub/bathtub_0001/point_cloud/iteration_15000/point_cloud.ply


  0%|          | 1/3991 [00:01<1:36:43,  1.45s/it]

...

classname: bathtub, path: /content/drive/MyDrive/3dgs/datasets/bathtub/bathtub_0074/point_cloud/iteration_15000/point_cloud.ply


  2%|▏         | 74/3991 [01:17<57:53,  1.13it/s]  

classname: bathtub, path: /content/drive/MyDrive/3dgs/datasets/bathtub/bathtub_0075/point_cloud/iteration_15000/point_cloud.ply


  2%|▏         | 75/3991 [01:18<1:02:42,  1.04it/s]

classname: bathtub, path: /content/drive/MyDrive/3dgs/datasets/bathtub/bathtub_0076/point_cloud/iteration_15000/point_cloud.ply
